# Energy system optimisation with oemof

### Import modules

In [1]:
import os
import pandas as pd

from oemof.solph import (Sink, Source, Transformer, Bus, Flow,
                         EnergySystem, Model)
import oemof.outputlib as outputlib

### Specify solver

In [2]:
solver = 'cbc'

### Create an energy system and load data

In [3]:
datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
energysystem = EnergySystem(timeindex=datetimeindex)
filename = 'input_data.csv'
data = pd.read_csv(filename, sep=",")

### Create components and add them to the energysystem

In [4]:
# resource buses
bus_coal = Bus(label='coal', balanced=False)
bus_gas = Bus(label='gas', balanced=False)
bus_oil = Bus(label='oil', balanced=False)
bus_lig = Bus(label='lignite', balanced=False)

In [5]:
# electricity and heat buses
bus_el = Bus(label='electricity')
bus_th = Bus(label='heat')

In [6]:
# an excess and a shortage variable can help to avoid infeasible problems
excess_el = Sink(label='excess_el', inputs={bus_el: Flow()})

shortage_el = Source(label='shortage_el',
                     outputs={bus_el: Flow(variable_costs=200)})

In [7]:
# Renewable feedin
wind = Source(label='wind', outputs={bus_el: Flow(
              actual_value=data['wind'], nominal_value=66.3, fixed=True)})

pv = Source(label='pv', outputs={bus_el: Flow(
            actual_value=data['pv'], nominal_value=65.3, fixed=True)})

In [8]:
# Electricity/heat demand
demand_el = Sink(label='demand_el', inputs={bus_el: Flow(
                 nominal_value=85, actual_value=data['demand_el'], fixed=True)})

demand_th = Sink(label='demand_th',
                 inputs={bus_th: Flow(nominal_value=40,
                                   actual_value=data['demand_th'],
                                   fixed=True)})

In [9]:
energysystem.add(bus_coal, bus_gas, bus_oil, bus_lig, bus_el, bus_th,
                 excess_el, shortage_el, wind, pv, demand_el, demand_th)

In [10]:
# power plants
pp_coal = Transformer(label='pp_coal',
                      inputs={bus_coal: Flow()},
                      outputs={bus_el: Flow(nominal_value=20.2, variable_costs=25)},
                      conversion_factors={bus_el: 0.39})

pp_lig = Transformer(label='pp_lig',
                     inputs={bus_lig: Flow()},
                     outputs={bus_el: Flow(nominal_value=11.8, variable_costs=19)},
                     conversion_factors={bus_el: 0.41})

pp_gas = Transformer(label='pp_gas',
                     inputs={bus_gas: Flow()},
                     outputs={bus_el: Flow(nominal_value=41, variable_costs=40)},
                     conversion_factors={bus_el: 0.50})

pp_oil = Transformer(label='pp_oil',
                     inputs={bus_oil: Flow()},
                     outputs={bus_el: Flow(nominal_value=5, variable_costs=50)},
                     conversion_factors={bus_el: 0.28})

energysystem.add(pp_coal, pp_lig, pp_gas, pp_oil)

In [11]:
# combined heat and power plant (chp)
chp = Transformer(label='pp_chp',
                  inputs={bus_gas: Flow()},
                  outputs={bus_el: Flow(nominal_value=30, variable_costs=42),
                           bus_th: Flow(nominal_value=40)},
                  conversion_factors={bus_el: 0.3, bus_th: 0.4})

In [12]:
# heat pump with a coefficient of performance (COP) of 3
b_heat_source = Bus(label='b_heat_source')
energysystem.add(b_heat_source)

energysystem.add(Source(label='heat_source', outputs={b_heat_source: Flow()}))

cop = 3
heat_pump = Transformer(label='heat_pump',
            inputs={bus_el: Flow(),
                    b_heat_source: Flow()},
            outputs={bus_th: Flow(nominal_value=10)},
            conversion_factors={bus_el: 1/3, b_heat_source: (cop-1)/cop})

energysystem.add(chp, heat_pump)

### Optimization

In [13]:
# create optimization model based on energy_system
optimization_model = Model(energysystem=energysystem)

# solve problem
optimization_model.solve(solver=solver)

{'Problem': [{'Name': 'unknown', 'Lower bound': 383538.2988, 'Upper bound': 383538.2988, 'Number of objectives': 1, 'Number of constraints': 2641, 'Number of variables': 4081, 'Number of nonzeros': 1344, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.02, 'Wallclock time': 0.02, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 224}}, 'Error rc': 0, 'Time': 0.031032323837280273}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Write results into energysystem.results object for later

In [14]:
energysystem.results['main'] = outputlib.processing.results(optimization_model)
energysystem.results['meta'] = outputlib.processing.meta_results(optimization_model)

In [15]:
string_results = outputlib.views.convert_keys_to_strings(energysystem.results['main'])

### Save results - Dump the energysystem (to ~/home/user/.oemof by default)
Specify path and filename if you do not want to overwrite

In [16]:
energysystem.dump(dpath=None, filename=None)

'Attributes dumped to: /home/jann/.oemof/dumps/es_dump.oemof'